**Shift notebook and other files one directory up for use**

In [1]:
import cv2
import numpy as np
from PIL import Image
import os
import inspect
from skimage.metrics import structural_similarity as ssim
from pathlib import Path
from utils._common import _to_uint8_rgb
from utils import modelLoader

# Carlini and Wagner $L_\infty$ Attack

In [2]:
from carlini_linf_attack import carlini_linf_attack

folder = Path("../to_attack")
image_exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp"}
image_paths = sorted(
    str(p) for p in folder.iterdir()
    if p.is_file() and p.suffix.lower() in image_exts
)
print(f"Found {len(image_paths)} images")

resnet50_Model = modelLoader.loadModel("resnet50", "utils/.models/resnet50.pth")
# densenet121_Model = modelLoader.loadModel("densenet121", ".models/densenet121.pth")

allSSIMScores = []
successfulAttacks = 0

# ==== configure output folder ====
adv_output_dir = Path("../adv_images")
adv_output_dir.mkdir(parents=True, exist_ok=True)

'''
def _finalize(orig_uint8: np.ndarray, x_adv01: np.ndarray) -> tuple:
    adv_uint8 = _to_uint8_rgb(x_adv01[0])  # (H_adv, W_adv, 3)

    # Resize original to match adv size if needed
    H_adv, W_adv = adv_uint8.shape[:2]
    if orig_uint8.shape[:2] != (H_adv, W_adv):
        import cv2
        orig_match = cv2.resize(orig_uint8, (W_adv, H_adv), interpolation=cv2.INTER_LINEAR)
    else:
        orig_match = orig_uint8

    noise = adv_uint8.astype(np.int16) - orig_match.astype(np.int16)
    return orig_match, noise, adv_uint8
'''

for p in image_paths:
    print("Processing:", p)
    try:
        ##################################### ATTACK METHOD ########################################
        
        orig_u8, adv = carlini_linf_attack(resnet50_Model, p)

        ############################################################################################

        adv = np.transpose(adv, (1, 2, 0))
        _adv = adv
        #print(orig_u8.shape)
        #print(adv.shape)
        if orig_u8.shape[:2] != adv.shape[:2]:
            _adv = cv2.resize(
                adv, (orig_u8.shape[1], orig_u8.shape[0]),
                interpolation=cv2.INTER_LINEAR
            )
        adv_u8 = _to_uint8_rgb(_adv)
        noise = adv_u8.astype(np.int16) - orig_u8.astype(np.int16)
    
        # Save adversarial image to output folder
        out_path = adv_output_dir / Path(p).name
        cv2.imwrite(str(out_path), cv2.cvtColor(adv_u8, cv2.COLOR_RGB2BGR))
    
        # Compute SSIM
        ssim_kwargs = {"data_range": 255}
        if "channel_axis" in inspect.signature(ssim).parameters:
            ssim_kwargs["channel_axis"] = -1
        else:
            ssim_kwargs["multichannel"] = True
    
        #print(ssim_kwargs)
        ssim_score = ssim(orig_u8, adv_u8, **ssim_kwargs)
        allSSIMScores.append(ssim_score)
        print(f"SSIM between original and adversarial: {ssim_score:.6f}")
        print(f"Saved adversarial image: {out_path}")
        print(" ")
        successfulAttacks += 1
        
    except Exception as e:
        print("Error processing:", p)
        print(e)
        continue


print("")
print("Average SSIM Score:", np.mean(allSSIMScores))
print("Successful Attacks:", successfulAttacks)

Found 2 images
[MODEL] Loading 'resnet50' weights from utils/.models/resnet50.pth ...
[MODEL] 'resnet50' ready on cuda:0
Processing: ../to_attack/img_0337.png


C&W L_inf:   0%|          | 0/1 [00:00<?, ?it/s]

SSIM between original and adversarial: 0.992852
Saved adversarial image: ../adv_images/img_0337.png
 
Processing: ../to_attack/img_0667.png


C&W L_inf:   0%|          | 0/1 [00:00<?, ?it/s]

SSIM between original and adversarial: 0.997221
Saved adversarial image: ../adv_images/img_0667.png
 

Average SSIM Score: 0.9950366083050868
Successful Attacks: 2
